### Import Dataset

In [50]:
import os
import cv2
import mahotas
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import skew ,kurtosis
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from pywt import dwt2

In [ ]:
data_dir = 'C:/Users\Ayatullah Ahmed/OneDrive/Desktop/archive (11)/Skin_Data'
cancer_train_dir = os.path.join(data_dir, 'Cancer', 'Training')
cancer_test_dir = os.path.join(data_dir, 'Cancer', 'Testing')
non_cancer_train_dir = os.path.join(data_dir, 'Non_Cancer', 'Training')
non_cancer_test_dir = os.path.join(data_dir, 'Non_Cancer', 'Testing')

cancer_train_files = [os.path.join(cancer_train_dir, f) for f in os.listdir(cancer_train_dir) if f.lower().endswith(('.jpg', '.jpeg'))]
non_cancer_train_files = [os.path.join(non_cancer_train_dir, f) for f in os.listdir(non_cancer_train_dir) if f.lower().endswith(('.jpg', '.jpeg'))]

cancer_test_files = [os.path.join(cancer_test_dir, f) for f in os.listdir(cancer_test_dir) if f.lower().endswith(('.jpg', '.jpeg'))]
non_cancer_test_files = [os.path.join(non_cancer_test_dir, f) for f in os.listdir(non_cancer_test_dir) if f.lower().endswith(('.jpg', '.jpeg'))]

train_labels_cancer = [1] * len(cancer_train_files)
train_labels_non_cancer = [0] * len(non_cancer_train_files)
test_labels_cancer = [1] * len(cancer_test_files)
test_labels_non_cancer = [0] * len(non_cancer_test_files)

train_files = cancer_train_files + non_cancer_train_files
train_labels = train_labels_cancer + train_labels_non_cancer
test_files = cancer_test_files + non_cancer_test_files
test_labels = test_labels_cancer + test_labels_non_cancer

print("Number of Cancer training files:", len(cancer_train_files))
print("Number of Non-Cancer training files:", len(non_cancer_train_files))
print("Number of Cancer testing files:", len(cancer_test_files))
print("Number of Non-Cancer testing files:", len(non_cancer_test_files))

plt.figure(figsize=(10, 10))

# Example Cancer training image
plt.subplot(2, 2, 1)
if cancer_train_files:
    img = mpimg.imread(cancer_train_files[1])
    plt.imshow(img)
    plt.title("Example Cancer Training Image")
    plt.axis('off')
else:
    plt.text(0.5, 0.5, "No Cancer training files", ha='center', va='center')
    plt.axis('off')

# Example Non-Cancer training image
plt.subplot(2, 2, 2)
if non_cancer_train_files:
    img = mpimg.imread(non_cancer_train_files[0])
    plt.imshow(img)
    plt.title("Example Non-Cancer Training Image")
    plt.axis('off')
else:
    plt.text(0.5, 0.5, "No Non-Cancer training files", ha='center', va='center')
    plt.axis('off')

# Example Cancer testing image
plt.subplot(2, 2, 3)
if cancer_test_files:
    img = mpimg.imread(cancer_test_files[0])
    plt.imshow(img)
    plt.title("Example Cancer Testing Image")
    plt.axis('off')
else:
    plt.text(0.5, 0.5, "No Cancer testing files", ha='center', va='center')
    plt.axis('off')

# Example Non-Cancer testing image
plt.subplot(2, 2, 4)
if non_cancer_test_files:
    img = mpimg.imread(non_cancer_test_files[0])
    plt.imshow(img)
    plt.title("Example Non-Cancer Testing Image")
    plt.axis('off')
else:
    plt.text(0.5, 0.5, "No Non-Cancer testing files", ha='center', va='center')
    plt.axis('off')

plt.tight_layout()
plt.show()

### Preprocessing

In [17]:
def remove_glare(image):
    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    _, s, v = cv2.split(hsv)  # Saturation and Value channels

    # Normalize the Value channel
    v = cv2.normalize(v, None, 0, 255, cv2.NORM_MINMAX)

    # Adaptive thresholding with Otsu's method
    _, glare_mask = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Refine with saturation mask (glare has low saturation)
    _, saturation_mask = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY_INV)
    glare_mask = cv2.bitwise_and(glare_mask, saturation_mask)

    # Dilate and erode to refine the mask
    kernel = np.ones((3, 3), np.uint8)
    glare_mask = cv2.dilate(glare_mask, kernel, iterations=1)
    glare_mask = cv2.erode(glare_mask, kernel, iterations=1)

    # Inpaint the glare regions
    image_no_glare = cv2.inpaint(image, glare_mask, 5, cv2.INPAINT_TELEA)

    # Check if glare mask is too large (likely over-segmentation)
    mask_area = np.sum(glare_mask == 255)
    image_area = image.shape[0] * image.shape[1]
    alpha = 0.7  # Default blending factor
    if mask_area / image_area > 0.3:  # If glare mask covers more than 30%
        alpha = 0.9  # Increase blending to preserve more details

    # Blend with the original image
    image_no_glare = cv2.addWeighted(image, alpha, image_no_glare, 1 - alpha, 0)

    return image_no_glare, glare_mask 

def remove_hair(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply black-hat morphological operation to detect dark hair
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # Threshold to create hair mask
    _, hair_mask = cv2.threshold(blackhat, 30, 255, cv2.THRESH_BINARY)

    # Inpaint the hair regions
    inpainted = cv2.inpaint(image, hair_mask, 3, cv2.INPAINT_TELEA)

    return inpainted, hair_mask

In [18]:
def visualize_processed_image(processed_img):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 5, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(processed_img['original'], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 5, 2)
    plt.title("Hair Mask")
    plt.imshow(processed_img['hair_mask'], cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 5, 3)
    plt.title("Hair Removed")
    plt.imshow(cv2.cvtColor(processed_img['hair_removed'], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 5, 4)
    plt.title("Glare Mask")
    plt.imshow(processed_img['glare_mask'], cmap='gray')
    plt.axis('off')
    
    plt.subplot(1, 5, 5)
    plt.title("All Processed")
    plt.imshow(cv2.cvtColor(processed_img['glare_removed'], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Additional function to visualize the fully processed image compared to original
def visualize_fully_processed(processed_img):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(cv2.cvtColor(processed_img['original'], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.title("Fully Processed (Hair & Glare Removed)")
    plt.imshow(cv2.cvtColor(processed_img['glare_removed'], cv2.COLOR_BGR2RGB))
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [19]:
def process_image(img_path):
    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to load {img_path}")
        return None
        
    # Remove hair
    img_hair_removed, hair_mask = remove_hair(img)

    # Remove glare on the hair-removed image
    img_fully_processed, glare_mask = remove_glare(img_hair_removed)
    
    return {
        'original': img,
        'hair_removed': img_hair_removed,
        'glare_removed': img_fully_processed,  # This is now both hair and glare removed
        'hair_mask': hair_mask,
        'glare_mask': glare_mask
    }

In [ ]:
# Process sample cancer training images
sample_size = min(10, len(cancer_train_files))
processed_train_cancer=[]
for i, img_path in enumerate(cancer_train_files[:sample_size]):
    print(f"Processing cancer training image {i+1}/{sample_size}")
    processed_img = process_image(img_path)
    if processed_img:
        processed_train_cancer.append(processed_img)
        visualize_processed_image(processed_img)  # Show all steps separately
        visualize_fully_processed(processed_img)  # Compare original vs fully processed



In [ ]:
# Process sample non-cancer training images
for i, img_path in enumerate(non_cancer_train_files[:sample_size]):
    processed_train_non_cancer=[]
    print(f"Processing non-cancer training image {i+1}/{sample_size}")
    processed_img = process_image(img_path)
    if processed_img:
        processed_train_non_cancer.append(processed_img)
        visualize_processed_image(processed_img)  # Show all steps separately
        visualize_fully_processed(processed_img)  # Compare original vs fully processed

# Process sample cancer test images


In [ ]:
for i, img_path in enumerate(cancer_test_files[:sample_size]):
    processed_test_cancer=[]
    print(f"Processing cancer test image {i+1}/{sample_size}")
    processed_img = process_image(img_path)
    if processed_img:
        processed_test_cancer.append(processed_img)
        visualize_processed_image(processed_img)  # Show all steps separately
        visualize_fully_processed(processed_img)  # Compare original vs fully processed



In [ ]:
# Process sample non-cancer test images
for i, img_path in enumerate(non_cancer_test_files[:sample_size]):
    processed_test_non_cancer=[]
    print(f"Processing non-cancer test image {i+1}/{sample_size}")
    processed_img = process_image(img_path)
    if processed_img:
        processed_test_non_cancer.append(processed_img)
        visualize_processed_image(processed_img)  # Show all steps separately
        visualize_fully_processed(processed_img)  # Compare original vs fully processed


### Feature Extraction

In [40]:
def extract_color_histogram(image, mask=None, bins=64):
    """Extract color histogram (192 features)."""
    if mask is not None:
        masked_image = image.copy()
        masked_image[mask == 0] = 0
    else:
        masked_image = image
    hist_r = np.histogram(masked_image[:, :, 0].ravel(), bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(masked_image[:, :, 1].ravel(), bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(masked_image[:, :, 2].ravel(), bins=bins, range=(0, 256))[0]
    hist_r = hist_r / (hist_r.sum() + 1e-10)
    hist_g = hist_g / (hist_g.sum() + 1e-10)
    hist_b = hist_b / (hist_b.sum() + 1e-10)
    return np.concatenate([hist_r, hist_g, hist_b])

def extract_color_moments(image, mask=None):
    """Extract color moments in RGB (mean, variance, skewness, kurtosis; 12 features)."""
    if mask is not None:
        masked_image = image.copy()
        masked_image[mask == 0] = 0
    else:
        masked_image = image
    moments = []
    for channel in range(3):
        pixels = masked_image[:, :, channel].ravel()
        pixels = pixels[pixels != 0] if mask is not None else pixels
        if len(pixels) == 0:
            moments.extend([0, 0, 0, 0])
            continue
        mean = np.mean(pixels)
        variance = np.var(pixels)
        skewness = skew(pixels, bias=False) if variance > 0 else 0
        kurt = kurtosis(pixels, bias=False) if variance > 0 else 0
        moments.extend([mean, variance, skewness, kurt])
    return np.array(moments)

def extract_hsv_stats(image, mask=None):
    """Extract mean and std of HSV channels (6 features)."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    if mask is not None:
        masked_hsv = hsv_image.copy()
        masked_hsv[mask == 0] = 0
    else:
        masked_hsv = hsv_image
    stats = []
    for channel in range(3):
        pixels = masked_hsv[:, :, channel].ravel()
        pixels = pixels[pixels != 0] if mask is not None else pixels
        if len(pixels) == 0:
            stats.extend([0, 0])
            continue
        mean = np.mean(pixels)
        std = np.std(pixels)
        stats.extend([mean, std])
    return np.array(stats)

def extract_lab_stats(image, mask=None):
    """Extract mean and std of LAB channels (6 features)."""
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    if mask is not None:
        masked_lab = lab_image.copy()
        masked_lab[mask == 0] = 0
    else:
        masked_lab = lab_image
    stats = []
    for channel in range(3):
        pixels = masked_lab[:, :, channel].ravel()
        pixels = pixels[pixels != 0] if mask is not None else pixels
        if len(pixels) == 0:
            stats.extend([0, 0])
            continue
        mean = np.mean(pixels)
        std = np.std(pixels)
        stats.extend([mean, std])
    return np.array(stats)

def extract_haralick_and_stats(image, mask=None, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """Extract Haralick features and intensity stats (mean, std, entropy, kurtosis; 17 features)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is not None:
        gray[mask == 0] = 0
    pixels = gray.ravel()
    pixels = pixels[pixels != 0] if mask is not None else pixels
    if len(pixels) == 0:
        stats = [0, 0, 0, 0]
    else:
        mean = np.mean(pixels)
        std = np.std(pixels)
        entropy = -np.sum([(p/pixels.sum()) * np.log2(p/pixels.sum() + 1e-10) 
                          for p in np.histogram(pixels, bins=256)[0] if p > 0])
        kurt = kurtosis(pixels, bias=False) if std > 0 else 0
        stats = [mean, std, entropy, kurt]
    if mask is not None:
        gray_for_glcm = gray.copy()
        gray_for_glcm[mask == 0] = 0
    else:
        gray_for_glcm = gray
    gray_for_glcm = gray_for_glcm.astype(np.uint8)
    glcm = graycomatrix(gray_for_glcm, distances=distances, angles=angles, 
                        levels=256, symmetric=True, normed=True)
    haralick_features = []
    properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    for prop in properties:
        prop_values = graycoprops(glcm, prop).mean(axis=1).ravel()
        haralick_features.extend(prop_values)
    haralick_mahotas = mahotas.features.haralick(gray_for_glcm, return_mean=True)
    extra_features = haralick_mahotas[[0, 1, 2, 3, 4, 8]]
    return np.concatenate([haralick_features, extra_features, stats])

def extract_wavelet_features(image, mask=None, wavelet='db1'):
    """Extract wavelet transform features (mean and std of coefficients; 6 features)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is not None:
        gray[mask == 0] = 0
    
    coeffs = dwt2(gray, wavelet)
    cA, (cH, cV, cD) = coeffs  
    features = []
    for coeff in [cH, cV, cD]:  
        pixels = coeff.ravel()
        pixels = pixels[pixels != 0] if mask is not None else pixels
        if len(pixels) == 0:
            features.extend([0, 0])
        else:
            mean = np.mean(pixels)
            std = np.std(pixels)
            features.extend([mean, std])
    return np.array(features)

def extract_lbp(image, mask=None, radius=3, n_points=24):
    """Extract LBP histogram (26 features)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is not None:
        gray[mask == 0] = 0
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), 
                          range=(0, n_points + 2), density=True)
    return hist

def extract_hu_moments(image, mask=None):
    """Extract Hu Moments (7 features)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is not None:
        gray[mask == 0] = 0
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).ravel()
    hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
    return hu_moments

def extract_diameter(image, mask=None):
    """Extract lesion diameter (1 feature)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is None:
        _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        diameter = 0
    else:
        contour = max(contours, key=cv2.contourArea)
        
        if len(contour) >= 5:  
            ellipse = cv2.fitEllipse(contour)
            diameter = max(ellipse[1])  
        else:
            
            area = cv2.contourArea(contour)
            diameter = 2 * np.sqrt(area / np.pi)
    return np.array([diameter])

def extract_circularity(image, mask=None):
    """Extract Circularity (1 feature)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is None:
        _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        circularity = 0
    else:
        contour = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        circularity = (4 * np.pi * area) / (perimeter ** 2 + 1e-10)
    return np.array([circularity])

def extract_asymmetry_and_border(image, mask=None):
    """Extract asymmetry and border irregularity (2 features)."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    if mask is None:
        _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask = mask.astype(np.uint8)
    flipped = cv2.flip(mask, 1)
    intersection = cv2.bitwise_and(mask, flipped)
    union = cv2.bitwise_or(mask, flipped)
    asymmetry_index = 1 - (np.sum(intersection) / (np.sum(union) + 1e-10))
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        border_irregularity = 0
    else:
        contour = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        border_irregularity = perimeter ** 2 / (4 * np.pi * area + 1e-10)
    return np.array([asymmetry_index, border_irregularity])

def normalize_features(features):
    """Normalize features to [0, 1] range."""
    if features.size == 0:
        return features
    min_val = np.min(features)
    max_val = np.max(features)
    if max_val == min_val:
        return np.zeros_like(features)
    return (features - min_val) / (max_val - min_val)


In [47]:
def extract_all_features(image, mask=None):
    """Extract all features (color, texture, shape) and normalize."""
    color_hist = extract_color_histogram(image, mask, bins=64)  # 192 features
    color_moments = extract_color_moments(image, mask)  # 12 features
    hsv_stats = extract_hsv_stats(image, mask)  # 6 features
    lab_stats = extract_lab_stats(image, mask)  # 6 features
    haralick_stats = extract_haralick_and_stats(image, mask)  # 17 features
    wavelet_features = extract_wavelet_features(image, mask)  # 6 features
    lbp_hist = extract_lbp(image, mask, radius=3, n_points=24)  # 26 features
    hu_moments = extract_hu_moments(image, mask)  # 7 features
    diameter = extract_diameter(image, mask)  # 1 feature
    circularity = extract_circularity(image, mask)  # 1 feature
    asym_border = extract_asymmetry_and_border(image, mask)  # 2 features

    features = np.concatenate([
        color_hist,
        color_moments,
        hsv_stats,
        lab_stats,
        haralick_stats,
        wavelet_features,
        lbp_hist,
        hu_moments,
        diameter,
        circularity,
        asym_border
    ])

    features = normalize_features(features)
    return features

In [48]:
def extract_and_save_features(train_files, test_files, train_labels, test_labels, output_path='features.npz'):
    """
    Process all images, extract features, and save to NPZ file.
    
    Parameters:
    - train_files: List of training image file paths
    - test_files: List of testing image file paths
    - train_labels: List of training image labels
    - test_labels: List of testing image labels
    - output_path: Path to save the NPZ file
    """
    print(f"Processing {len(train_files)} training images and {len(test_files)} test images...")
    
    
    train_features = []
    test_features = []
    
    
    for i, img_path in enumerate(train_files):
        if i % 10 == 0:
            print(f"Processing training image {i+1}/{len(train_files)}")
        
        
        processed_img = process_image(img_path)
        if processed_img:
            
            features = extract_all_features(processed_img['glare_removed'])
            train_features.append(features)
    
    
    for i, img_path in enumerate(test_files):
        if i % 10 == 0:
            print(f"Processing test image {i+1}/{len(test_files)}")
        
        
        processed_img = process_image(img_path)
        if processed_img:
            
            features = extract_all_features(processed_img['glare_removed'])
            test_features.append(features)
    
    
    train_features = np.array(train_features)
    test_features = np.array(test_features)
    
    np.savez(
        output_path,
        train_features=train_features,
        test_features=test_features,
        train_labels=np.array(train_labels[:len(train_features)]),
        test_labels=np.array(test_labels[:len(test_features)])
    )
    
    print(f"Features extracted and saved to {output_path}")
    print(f"Training features shape: {train_features.shape}")
    print(f"Testing features shape: {test_features.shape}")
    
    return train_features, test_features


In [ ]:
features_file = "kaggle/output/skin_cancer_features.npz"
if not os.path.exists(features_file):
    os.makedirs("kaggle/output", exist_ok=True)
    
    # Extract and save features
    train_features, test_features = extract_and_save_features(
        train_files, 
        test_files, 
        train_labels, 
        test_labels,
        output_path=features_file
    )
else:
    print(f"Loading features from {features_file}")
    data = np.load(features_file)
    train_features = data['train_features']
    test_features = data['test_features']
    train_labels = data['train_labels']
    test_labels = data['test_labels']

print(f"Loaded features: {train_features.shape} training samples, {test_features.shape} test samples")

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

# Split training data for validation
X_train, X_val, y_train, y_val = train_test_split(
    train_features_scaled, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {test_features_scaled.shape}")

### Models setup

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Define parameter grids
svm_params = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear'],
    'gamma': ['scale', 'auto']
}

rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Grid search for SVM
svm = SVC(probability=True)
svm_grid = GridSearchCV(svm, svm_params, cv=3, scoring='accuracy', n_jobs=-1)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_
print("Best SVM params:", svm_grid.best_params_)

# Grid search for Random Forest
rf = RandomForestClassifier()
rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='accuracy', n_jobs=-1)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_
print("Best Random Forest params:", rf_grid.best_params_)

# Create ensemble model
ensemble = VotingClassifier(estimators=[
    ('svm', best_svm),
    ('rf', best_rf)
], voting='soft')  # soft voting uses predicted probabilities

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Evaluate on validation set
y_val_pred = ensemble.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate on test set
y_test_pred = ensemble.predict(test_features_scaled)
print("Test Accuracy:", accuracy_score(test_labels, y_test_pred))
print("Test Classification Report:")
print(classification_report(test_labels, y_test_pred))


### Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score
# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, title, ax):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
                xticklabels=['Non-Cancer', 'Cancer'], 
                yticklabels=['Non-Cancer', 'Cancer'], ax=ax)
    ax.set_title(title)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')

# Function to plot ROC curve
def plot_roc_curve(y_true, y_prob, title, ax):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(title)
    ax.legend(loc='lower right')

# Function to plot precision-recall curve
def plot_precision_recall_curve(y_true, y_prob, title, ax):
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    ax.plot(recall, precision, label=f'PR curve (AP = {avg_precision:.3f})')
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_title(title)
    ax.legend(loc='lower left')

# Evaluate model performance
def evaluate_model(model, X_train, y_train, X_val, y_val, X_test, y_test, feature_names=None):
    # Predictions and probabilities
    y_val_pred = model.predict(X_val)
    y_val_prob = model.predict_proba(X_val)[:, 1]
    y_test_pred = model.predict(X_test)
    y_test_prob = model.predict_proba(X_test)[:, 1]

    # Print classification reports
    print("Validation Classification Report:")
    print(classification_report(y_val, y_val_pred, target_names=['Non-Cancer', 'Cancer']))
    print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.3f}")
    print(f"Validation ROC-AUC: {auc(roc_curve(y_val, y_val_prob)[0], roc_curve(y_val, y_val_prob)[1]):.3f}")
    print("\nTest Classification Report:")
    print(classification_report(y_test, y_test_pred, target_names=['Non-Cancer', 'Cancer']))
    print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
    print(f"Test ROC-AUC: {auc(roc_curve(y_test, y_test_prob)[0], roc_curve(y_test, y_test_prob)[1]):.3f}")

    # Cross-validation
    print("\nPerforming 5-fold Cross-Validation on Training Data...")
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"Cross-Validation Accuracy: {np.mean(cv_scores):.3f} ± {np.std(cv_scores):.3f}")

    # Visualizations
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    # Confusion matrices
    plot_confusion_matrix(y_val, y_val_pred, "Validation Confusion Matrix", axes[0, 0])
    plot_confusion_matrix(y_test, y_test_pred, "Test Confusion Matrix", axes[0, 1])

    # ROC curves
    plot_roc_curve(y_val, y_val_prob, "Validation ROC Curve", axes[0, 2])
    plot_roc_curve(y_test, y_test_prob, "Test ROC Curve", axes[1, 2])

    # Precision-Recall curves
    plot_precision_recall_curve(y_val, y_val_prob, "Validation Precision-Recall Curve", axes[1, 0])
    plot_precision_recall_curve(y_test, y_test_prob, "Test Precision-Recall Curve", axes[1, 1])

    plt.tight_layout()
    plt.show()

    # Feature importance (for Random Forest component)
    if hasattr(model, 'estimators_') and isinstance(model.estimators_[1], RandomForestClassifier):
        rf = model.estimators_[1]  # Random Forest is the second estimator in the VotingClassifier
        importances = rf.feature_importances_
        indices = np.argsort(importances)[::-1][:10]  # Top 10 features
        if feature_names is not None:
            top_features = [feature_names[i] for i in indices]
        else:
            top_features = [f"Feature {i}" for i in indices]
        
        plt.figure(figsize=(10, 6))
        sns.barplot(x=importances[indices], y=top_features)
        plt.title("Top 10 Feature Importances (Random Forest)")
        plt.xlabel("Importance")
        plt.tight_layout()
        plt.show()

    # Error analysis: Identify misclassified test samples
    misclassified_idx = np.where(y_test != y_test_pred)[0]
    print(f"\nNumber of misclassified test samples: {len(misclassified_idx)}")
    if len(misclassified_idx) > 0:
        print("Sample misclassified test images (indices):", misclassified_idx[:5])
        # Optionally, visualize a few misclassified images
        plt.figure(figsize=(15, 5))
        for i, idx in enumerate(misclassified_idx[:3]):  # Show up to 3 misclassified images
            img = cv2.imread(test_files[idx])
            if img is not None:
                plt.subplot(1, 3, i+1)
                plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                plt.title(f"True: {['Non-Cancer', 'Cancer'][y_test[idx]]}\nPred: {['Non-Cancer', 'Cancer'][y_test_pred[idx]]}")
                plt.axis('off')
        plt.tight_layout()
        plt.show()

feature_names = (
    [f"Color_Hist_R_{i}" for i in range(64)] +
    [f"Color_Hist_G_{i}" for i in range(64)] +
    [f"Color_Hist_B_{i}" for i in range(64)] +
    [f"Color_Moment_{m}_{c}" for c in ['R', 'G', 'B'] for m in ['Mean', 'Variance', 'Skewness', 'Kurtosis']] +
    [f"HSV_{s}_{c}" for c in ['H', 'S', 'V'] for s in ['Mean', 'Std']] +
    [f"LAB_{s}_{c}" for c in ['L', 'A', 'B'] for s in ['Mean', 'Std']] +
    [f"Haralick_{p}_{i}" for p in ['Contrast', 'Dissimilarity', 'Homogeneity', 'Energy', 'Correlation', 'ASM'] for i in range(1, 5)] +
    [f"Haralick_Mahotas_{i}" for i in [0, 1, 2, 3, 4, 8]] +
    ['Intensity_Mean', 'Intensity_Std', 'Intensity_Entropy', 'Intensity_Kurtosis'] +
    [f"Wavelet_{c}_{s}" for c in ['H', 'V', 'D'] for s in ['Mean', 'Std']] +
    [f"LBP_{i}" for i in range(26)] +
    [f"Hu_Moment_{i}" for i in range(7)] +
    ['Diameter', 'Circularity', 'Asymmetry', 'Border_Irregularity']
)
evaluate_model(ensemble, X_train, y_train, X_val, y_val, test_features_scaled, test_labels, feature_names=feature_names)